# Modelo 2: Clustering de jugadoras por criterios

## Librerías necesarias para la ejecución

In [4]:
# Librerias de sistema
import os
from dotenv import load_dotenv
# Librerias de tratado de datos
import pandas as pd
import numpy as np
# Librerias para modelos
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
import mlflow
import mlflow.sklearn


### Carga y preprocesado de datos

In [5]:
def load_dataset(path: str, sep = ";") -> pd.DataFrame:
    return pd.read_csv(path, sep=sep)

In [6]:
def preprocess(df: pd.DataFrame, id_cols=('Nombre','Temporada','Equipo')):
    X = df.drop(columns=list(id_cols), errors="ignore")
    X = X.select_dtypes(include=[np.number])

    pipe = SimpleImputer(strategy="median")
    X = pipe.fit_transform(X)

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    return X, df[list(id_cols)]

### Algoritmos de clustering

In [7]:
def kmeans_run(X, k, pca_n=None):
    X_fit = PCA(n_components=pca_n, random_state=42).fit_transform(X) if pca_n else X
    model = KMeans(n_clusters=k, n_init="auto", random_state=42)
    labels = model.fit_predict(X_fit)
    sil = silhouette_score(X_fit, labels)
    return model, labels, sil

In [8]:
def dbscan_run(X, eps, min_samples, pca_n=None):
    X_fit = PCA(n_components=pca_n, random_state=42).fit_transform(X) if pca_n else X
    model = DBSCAN(eps=eps, min_samples=min_samples)
    labels = model.fit_predict(X_fit)
    # silueta ignorando ruido (-1)
    valid = labels != -1
    sil = silhouette_score(X_fit[valid], labels[valid]) if valid.sum() >= 2 else -1
    return model, labels, sil

## MLFlow

In [ ]:
def main(csv_path, tracking_uri, experiment):

    # MLflow
    mlflow.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(experiment)

    # Datos
    df = load_dataset(csv_path, sep=";")
    X, meta = preprocess(df)

    results: dict[str, float] = {}

    # ----- K-Means -----
    for k in range(2, 11):
        with mlflow.start_run(run_name=f"KMeans_k={k}"):
            m, y, sil = kmeans_run(X, k)
            mlflow.log_params({"algorithm": "kmeans", "k": k})
            mlflow.log_metric("silhouette", sil)
            mlflow.sklearn.log_model(m, "model")
            results[f"kmeans_{k}"] = sil
            print(f"KMeans k={k}  →  silhouette={sil:.3f}")

    # ----- PCA + K-Means -----
    for k in range(2, 11):
        for n_comp in (2, 3, 5):
            with mlflow.start_run(run_name=f"PCA{k}_n{n_comp}"):
                m, y, sil = kmeans_run(X, k, pca_n=n_comp)
                mlflow.log_params({"algorithm": "pca+kmeans",
                                   "k": k,
                                   "pca_components": n_comp})
                mlflow.log_metric("silhouette", sil)
                mlflow.sklearn.log_model(m, "model")
                results[f"pca{k}_{n_comp}"] = sil
                print(f"PCA({n_comp}) + KMeans k={k}  →  silhouette={sil:.3f}")

    # ----- DBSCAN -----
    for eps in (0.5, 1.0, 1.5):
        for ms in (3, 5, 10):
            with mlflow.start_run(run_name=f"DBSCAN_eps{eps}_ms{ms}"):
                m, y, sil = dbscan_run(X, eps, ms, pca_n=3)
                mlflow.log_params({"algorithm": "dbscan",
                                   "eps": eps,
                                   "min_samples": ms})
                mlflow.log_metric("silhouette", sil)
                mlflow.sklearn.log_model(m, "model")
                results[f"dbscan_{eps}_{ms}"] = sil
                print(f"DBSCAN eps={eps} min_samples={ms}  →  silhouette={sil:.3f}")
                
    # Selección del mejor modelo 
    best_key = max(results, key=results.get)
    print(f"\nMejor experimento: {best_key}  |  silhouette = {results[best_key]:.3f}")

    # Recreamos el modelo ganador para etiquetar todas las jugadoras
    if best_key.startswith("kmeans_"):
        k = int(best_key.split("_")[1])
        best_model, labels, _ = kmeans_run(X, k)
    elif best_key.startswith("pca"):
        k = int(best_key[3:].split("_")[0])
        n_comp = int(best_key.split("_")[1])
        best_model, labels, _ = kmeans_run(X, k, pca_n=n_comp)
    else:  # DBSCAN
        _, eps, ms = best_key.split("_")
        best_model, labels, _ = dbscan_run(X, float(eps), int(ms), pca_n=3)

    # ---------- 5. Exportar resultados ----------
    resultado = meta.copy()
    resultado["cluster"] = labels
    salida = "jugadoras_clusterizadas.csv"
    resultado.to_csv(salida, index=False, sep=";")
    print(f"Etiquetas guardadas en {salida}")

In [10]:
if __name__ == "__main__":
    load_dotenv()
    main(csv_path=os.getenv("TRANSFORM_DATA_PATH"), tracking_uri="mlruns",experiment="Clustering_Jugadoras")

2025/06/15 18:26:30 INFO mlflow.tracking.fluent: Experiment with name 'Clustering_Jugadoras' does not exist. Creating a new experiment.
2025/06/15 18:26:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/15 18:26:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


KMeans k=2  →  silhouette=0.315


2025/06/15 18:26:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/15 18:26:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


KMeans k=3  →  silhouette=0.254


2025/06/15 18:26:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/15 18:26:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

2025/06/15 18:29:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/15 18:29:13 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/06/15 18:29:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


DBSCAN eps=0.5 min_samples=10  →  silhouette=0.199


2025/06/15 18:29:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/15 18:29:18 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/06/15 18:29:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


DBSCAN eps=1.0 min_samples=3  →  silhouette=0.382


ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)